In [ ]:
#This Notebook is based on code from Eric Chen for generating speckle noise data sets

In [ ]:
import numpy as np
import pickle
import os

In [ ]:
#Create the speckle noise datasets (no additive noise)
data_path  = "/Users/sarahhagan/Desktop/NeRP Research/ILSVRC2012_50x50_small.pkl"  # path to your .pkl of shape (N,1024)
out_dir    = "/Users/sarahhagan/Desktop/NeRP Research/Speckle_noise_datasets"      # where to dump the .npz files
threshold  = 1.0
L          = 50
rates      = [0.1, 0.2, 0.3, 0.4]
sigma_w    = 1.0
with open(data_path, "rb") as f:
    data = pickle.load(f)        
N, n = data.shape
print(f"loaded {N} patches of dimension {n}")
mask = np.max(data, axis=1) > threshold
X = data[mask]
M = X.shape[0]
print(f"keeping {M} patches (max‐pixel > {threshold})")
X /= 255
print(X)
os.makedirs(out_dir, exist_ok=True)
rng = np.random.default_rng(0)
for r in rates:
    m = int(r * n)
    A = rng.standard_normal((m, n)) / np.sqrt(m)
    Y = np.zeros((M, L, m), dtype=float)
    # generate looks
    for i, x in enumerate(X):
        rng2 = np.random.default_rng(i)
        W_block = rng2.standard_normal((L, n)) * sigma_w
        X_block = x[None, :] * W_block
        Y[i] = X_block.dot(A.T)
    out_path = os.path.join(out_dir, f"coherent_{int(r*100)}pct.npz")
    np.savez_compressed(out_path, A=A, X=X, Y=Y)
    print(f"→ saved {out_path}: A({m},{n}), X({M},{n}), Y({M},{L},{m})")

In [ ]:
#Create the speckle noise datasets with no sensing matrix and no additive noise
data_path  = "/Users/sarahhagan/Desktop/NeRP Research/ILSVRC2012_50x50_small.pkl"  # path to your .pkl of shape (N,1024)
out_dir    = "/Users/sarahhagan/Desktop/NeRP Research/Speckle_noise_datasets"      # where to dump the .npz files
threshold  = 1.0
L          = 50
sigma_w    = 1.0
with open(data_path, "rb") as f:
    data = pickle.load(f)   
N, n = data.shape
print(f"loaded {N} patches of dimension {n}")
mask = np.max(data, axis=1) > threshold
X = data[mask]
M = X.shape[0]
print(f"keeping {M} patches (max‐pixel > {threshold})")
X /= 255
print(X)
os.makedirs(out_dir, exist_ok=True)
m = n
A = np.identity(n, dtype=float)
Y = np.zeros((M, L, m), dtype=float)
# generate looks
for i, x in enumerate(X):
    rng2 = np.random.default_rng(i)
    W_block = rng2.standard_normal((L, n)) * sigma_w
    X_block = x[None, :] * W_block
    Y[i] = X_block.dot(A.T)
out_path = os.path.join(out_dir, f"coherent_just_speckle_imagenet50.npz")
print("path created")
np.savez_compressed(out_path, A=A, X=X, Y=Y)
print("saved")
print(f"→ saved {out_path}: A({m},{n}), X({M},{n}), Y({M},{L},{m})")

In [ ]:
# Example for Loading the data
data = np.load("Speckle_noise_datasets/coherent_10pct.npz")
A_loaded = data["A"] # m * n
X_loaded = data["X"] # N * n
Y_loaded = data["Y"] # N * L * m
m = A_loaded.shape[0]
n = A_loaded.shape[1]
N = X_loaded.shape[0]
L = Y_loaded.shape[1]
print("m =",m,"n =",n,"N =",N,"L =",L)